In [1]:
!pip install langchain-openai langchain

import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

In [2]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ce12640e350>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ce12640f7f0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

## Working with a prompt template

In [3]:
from langchain.prompts import ChatPromptTemplate


template = """
            Extract the following details from a description of a company:


            name: the name of the company
            sector: the sector it is in.
            valuation: its valuation in dollars $

            Format the output as JSON with the following keys:

            name
            sector
            valuation

            Company description:

            {company_description}

            """
prompt_template = ChatPromptTemplate.from_template(template)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['company_description'], template='\n            Extract the following details from a description of a company:\n\n\n            name: the name of the company\n            sector: the sector it is in.\n            valuation: its valuation in dollars $\n\n            Format the output as JSON with the following keys:\n\n            name\n            sector\n            valuation\n\n            Company description:\n\n            {company_description}\n\n            ')

In [4]:
prompt_template.messages[0].prompt.input_variables

['company_description']

In [5]:
company_description = """ EcoFusion Technologies is an innovative start-up breaking new grounds in the clean energy sector.
                          The company focuses on designing and manufacturing highly efficient fusion energy generators that promote green and sustainable energy.
                          Through the fusion of cutting-edge technology and commitment to environmental preservation, EcoFusion Technologies is pushing the boundaries of
                          what's possible in the renewable energy landscape.

                          Current Valuation: As of our latest funding round, EcoFusion Technologies is valued at $1.2 billion, demonstrating significant investor confidence
                          in our future growth prospects.

                      """


filled_in_prompt = prompt_template.format_messages(company_description=company_description)
filled_in_prompt

[HumanMessage(content="\n            Extract the following details from a description of a company:\n\n\n            name: the name of the company\n            sector: the sector it is in.\n            valuation: its valuation in dollars $\n\n            Format the output as JSON with the following keys:\n\n            name\n            sector\n            valuation\n\n            Company description:\n\n             EcoFusion Technologies is an innovative start-up breaking new grounds in the clean energy sector.\n                          The company focuses on designing and manufacturing highly efficient fusion energy generators that promote green and sustainable energy.\n                          Through the fusion of cutting-edge technology and commitment to environmental preservation, EcoFusion Technologies is pushing the boundaries of\n                          what's possible in the renewable energy landscape.\n\n                          Current Valuation: As of our latest fund

In [6]:
response = chat.invoke(filled_in_prompt).content
print(response)

{
    "name": "EcoFusion Technologies",
    "sector": "clean energy",
    "valuation": "$1.2 billion"
}


In [7]:
type(response)

str

## Formatting & Parsing The Output Using LangChain Response Schemas & Output Parser

In [8]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


name_schema = ResponseSchema(name="name",
                             description="The name of the company.")
sector_schema = ResponseSchema(name="sector",
                                      description="The sector the company is in.")
valuation = ResponseSchema(name="valuation",
                                    description="The valution of the company in dollars ($)")

response_schemas = [name_schema,
                    sector_schema,
                    valuation]
response_schemas

[ResponseSchema(name='name', description='The name of the company.', type='string'),
 ResponseSchema(name='sector', description='The sector the company is in.', type='string'),
 ResponseSchema(name='valuation', description='The valution of the company in dollars ($)', type='string')]

In [9]:
# Build an output parser using the response schema

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='name', description='The name of the company.', type='string'), ResponseSchema(name='sector', description='The sector the company is in.', type='string'), ResponseSchema(name='valuation', description='The valution of the company in dollars ($)', type='string')])

## Using the output parser to create automated format instructions

In [10]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // The name of the company.
	"sector": string  // The sector the company is in.
	"valuation": string  // The valution of the company in dollars ($)
}
```


In [15]:
template = """
            Extract the following details from a description of a company:


            name: the name of the company
            sector: the sector it is in.
            valuation: its valuation in dollars $

            company description: {company_description}

            Do not include the comments for the fields in your JSON output.

            {format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)

messages = prompt.format_messages(company_description=company_description,
                                format_instructions=format_instructions)

print(messages[0].content)


            Extract the following details from a description of a company:


            name: the name of the company
            sector: the sector it is in.
            valuation: its valuation in dollars $

            company description:  EcoFusion Technologies is an innovative start-up breaking new grounds in the clean energy sector.
                          The company focuses on designing and manufacturing highly efficient fusion energy generators that promote green and sustainable energy.
                          Through the fusion of cutting-edge technology and commitment to environmental preservation, EcoFusion Technologies is pushing the boundaries of
                          what's possible in the renewable energy landscape.

                          Current Valuation: As of our latest funding round, EcoFusion Technologies is valued at $1.2 billion, demonstrating significant investor confidence
                          in our future growth prospects.

              

In [16]:
response = chat.invoke(messages)
print(response.content)

```json
{
	"name": "EcoFusion Technologies",
	"sector": "clean energy",
	"valuation": "$1.2 billion"
}
```


## Use Output Parser to parse the JSON output

In [17]:
parsed_output = output_parser.parse(response.content)
print(parsed_output)

{'name': 'EcoFusion Technologies', 'sector': 'clean energy', 'valuation': '$1.2 billion'}


In [33]:
type(parsed_output)

dict

In [18]:
parsed_output['sector']

'clean energy'